<a href="https://colab.research.google.com/github/RozenkovAndrey/ABC-analysis/blob/main/ABC_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Выявление нерентабельных покупателей (ABC - анализ) по количеству купленных товаров, сумме продаж и среднему чеку**

**Описание проекта:**

Цель данного проекта - посмотреть динамику в росте/снижении продаж среди покупателей и по категория клиентов и росту/снижению продаж дать оценку рентабельности работы с ними.

На основании данных компании X за периоды с 01.10.2022 по 30.10.2022 (далее - предыдущий период) и с 01.11.2022 по 01.11.2022 (далее - текущий период) по прожажам покупателей необходимо подготовить отчёт для руководства.

**Требования к выполнению:**

- Провести анализ АВС 60/30/10 с расчетом доли и категории
- Сделать сравнительный расчет по отклонениям
- На основании расчета по отклонениям указать словами где «рост», а где «снижение»
- Создать отдельный расчет по покупателя «3», «5»
- Построить сравнительный график продаж текущего и предыдущего периода
- Проверить каких расчетов не хватает в отчете и добавить

Условия:
- Большое внимание уделить визуализации данных
- При проведении АВС анализа использовать цветовую гамму
- Необходимо использовать форматированную таблицу
- Отдельный расчет по группам «3» и «5» должен считаться автоматически (прописать формулы)
- Назначить срезы для удобства сортировки руководством
- Отчет должен помещаться на формат А4
- График не должен входить в отчет

In [49]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_excel('/content/drive/MyDrive/тестовое задание.xlsx')

In [51]:
#Удаляю пустые столбцы
df = df.drop('Unnamed: 1', axis = 1)
df = df.drop('Unnamed: 2', axis = 1)
df.head()

,Покупатель,Количество за текущий период,"Сумма, руб. за текущий период",Количество за предыдущий период,"Сумма, руб. за предыдущий период"
0,1,3417.05,27773.375212,3050.00,24219.586006
1,2,1790.83,19393.898652,1269.13,14298.177932
2,3-1,970.49,15494.945929,968.72,15657.027745
3,4,474.27,14876.846732,420.81,11213.582132
4,5-1,498.00,6369.915271,329.28,4316.667889


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Покупатель                        25 non-null     object 
 1   Количество за текущий период      25 non-null     float64
 2   Сумма, руб. за текущий период     25 non-null     float64
 3   Количество за предыдущий период   25 non-null     float64
 4   Сумма, руб. за предыдущий период  25 non-null     float64
dtypes: float64(4), object(1)
memory usage: 1.1+ KB


**Вывод:** Форматы данных соответствуют. Присутствуют дробные значения для количества, возможно, некоторые товары продаются в кг или дробном эквиваленте.

In [53]:
# Нахожу общее количество проданной продукции за текущий период
sum_count_now_period = df['Количество за текущий период'].sum()
sum_count_now_period

8968.3975

In [54]:
# Нахожу общую сумму по проданным товарам за текущий период
sum_money_now_period = df['Сумма, руб. за текущий период'].sum()
sum_money_now_period

112860.77640923078

In [55]:
# sum_count_last_period = df['Количество за предыдущий период'].sum()
# sum_count_last_period

In [56]:
# sum_money_last_period = df['Сумма, руб. за предыдущий период'].sum()
# sum_money_last_period

In [57]:
# Нахожу долю в процентах от общего количества за текущий период для покупателей
df['% от общего количества за текущий период'] = round(df['Количество за текущий период']*100/sum_count_now_period,2)
col = df.pop("% от общего количества за текущий период")
df.insert(int(df.columns.get_indexer(['Количество за текущий период']))+1, col.name, col)

In [58]:
# Нахожу долю в процентах от общей суммы продаж за текущий период для покупателей
df['% от общей суммы за текущий период'] = round(df['Сумма, руб. за текущий период']*100/sum_money_now_period,2)
col = df.pop("% от общей суммы за текущий период")
df.insert(int(df.columns.get_indexer(['Сумма, руб. за текущий период']))+1, col.name, col)

In [59]:
#Вывожу топ-5 покупателей по доле купленного количества продукции
df1 = df[['Покупатель', '% от общего количества за текущий период']].sort_values(by = '% от общего количества за текущий период', ascending = False)
df1.head()

,Покупатель,% от общего количества за текущий период
0,1,38.10
1,2,19.97
2,3-1,10.82
4,5-1,5.55
3,4,5.29


In [60]:
# Создаю временный столбец с накопленной кумулятивной суммой доли от общего количества продаж по покупателям
df1['с накоплением'] = np.cumsum(df1['% от общего количества за текущий период'])

In [61]:
def alert(row):
  if row['с накоплением'] < 60:
    return 'a'
  elif 60 <= row['с накоплением'] < 90:
    return 'b'
  else:
    return 'c'

In [62]:
# Создаю столбец с категориями a,b и с по по количеству продаж и покупателям
df1['категория количество за текущий период'] = df1.apply(alert, axis=1)

In [63]:
# Пересохраняю датафрейм со столбцами покупатель и категорией покупателя по количеству за текущий период
df1 = df1[['Покупатель', 'категория количество за текущий период']]

In [64]:
# Присоединяю столбец с категорией покупателя к начальному датафрейму
df = df.merge(df1, on = 'Покупатель', how = 'outer')

In [65]:
# Создаю датафрейм со столбцами покупатель и его категория по сумме продаж за текущий период
df2 = df[['Покупатель', '% от общей суммы за текущий период']].sort_values(by = '% от общей суммы за текущий период', ascending = False)
df2['с накоплением'] = np.cumsum(df2['% от общей суммы за текущий период'])
df2['категория сумма за текущий период'] = df2.apply(alert, axis=1)
df2 = df2[['Покупатель', 'категория сумма за текущий период']]

In [66]:
#Добавляю к датафрейму столбец с категорией покупателя по сумме продаж за текущий период
df = df.merge(df2, on = 'Покупатель', how = 'outer')

In [67]:
#Добавил в датафрейм столбец со средним чеком за текущий период
df['средний чек за текущий период'] = round(df['Сумма, руб. за текущий период']/df['Количество за текущий период'],2)
col = df.pop("средний чек за текущий период")
df.insert(int(df.columns.get_indexer(['% от общей суммы за текущий период']))+1, col.name, col)

In [68]:
# Считаю сумму средних чеков за текущий период
sum_avg_check_now = df['средний чек за текущий период'].sum()
sum_avg_check_now

588.7299999999999

In [69]:
# Добавляю столбец со средним чеком за текущий период
df['% от суммы средних чеков за текущий период'] = round(df['средний чек за текущий период']*100/sum_avg_check_now,2)
col = df.pop("% от суммы средних чеков за текущий период")
df.insert(int(df.columns.get_indexer(['средний чек за текущий период']))+1, col.name, col)

In [70]:
# Добавляю столбец с долей среднего чека от общего суммы средних чеков за текущий период
df3 = df[['Покупатель', '% от суммы средних чеков за текущий период']].sort_values(by = '% от суммы средних чеков за текущий период', ascending = False)
df3['с накоплением'] = np.cumsum(df3['% от суммы средних чеков за текущий период'])
df3['категория средний чек за текущий период'] = df3.apply(alert, axis=1)
df3 = df3[['Покупатель', 'категория средний чек за текущий период']]

In [71]:
df = df.merge(df3, on = 'Покупатель', how = 'outer')

In [72]:
# Провожу сортировку по покупателям
df['Покупатель'] = df['Покупатель'].astype('str')

In [73]:
df.sort_values(by = 'Покупатель', ascending = False)

,Покупатель,Количество за текущий период,% от общего количества за текущий период,"Сумма, руб. за текущий период",% от общей суммы за текущий период,средний чек за текущий период,% от суммы средних чеков за текущий период,Количество за предыдущий период,"Сумма, руб. за предыдущий период",категория количество за текущий период,категория сумма за текущий период,категория средний чек за текущий период
8,9,97.4575,1.09,2076.520917,1.84,21.31,3.62,103.58,2287.638043,c,b,b
7,8,242.3200,2.70,2134.526597,1.89,8.81,1.50,255.76,2234.235089,b,b,c
6,7,259.7200,2.90,3065.124412,2.72,11.80,2.00,239.80,2878.232071,b,b,b
5,6,76.1300,0.85,4296.925308,3.81,56.44,9.59,62.45,3432.886846,c,b,a
20,5-3,19.1900,0.21,734.662388,0.65,38.28,6.50,18.60,653.895818,c,c,a
12,5-2,71.0600,0.79,1817.693154,1.61,25.58,4.34,45.55,1069.295120,c,c,b
4,5-1,498.0000,5.55,6369.915271,5.64,12.79,2.17,329.28,4316.667889,b,b,b
3,4,474.2700,5.29,14876.846732,13.18,31.37,5.33,420.81,11213.582132,b,b,a
22,3-4,15.8400,0.18,500.884717,0.44,31.62,5.37,2.00,64.358969,c,c,a
18,3-3,14.1000,0.16,823.283538,0.73,58.39,9.92,0.00,0.000000,c,c,a


In [74]:
# Добавляю столбец с разницой сумм между периодами
df['Разница в суммах между текущим и прошедшим периодами'] = df['Сумма, руб. за текущий период'] - df['Сумма, руб. за предыдущий период']
df.head()

,Покупатель,Количество за текущий период,% от общего количества за текущий период,"Сумма, руб. за текущий период",% от общей суммы за текущий период,средний чек за текущий период,% от суммы средних чеков за текущий период,Количество за предыдущий период,"Сумма, руб. за предыдущий период",категория количество за текущий период,категория сумма за текущий период,категория средний чек за текущий период,Разница в суммах между текущим и прошедшим периодами
0,1,3417.05,38.10,27773.375212,24.61,8.13,1.38,3050.00,24219.586006,a,a,c,3553.789206
1,2,1790.83,19.97,19393.898652,17.18,10.83,1.84,1269.13,14298.177932,a,a,c,5095.720720
2,3-1,970.49,10.82,15494.945929,13.73,15.97,2.71,968.72,15657.027745,b,a,b,-162.081815
3,4,474.27,5.29,14876.846732,13.18,31.37,5.33,420.81,11213.582132,b,b,a,3663.264600
4,5-1,498.00,5.55,6369.915271,5.64,12.79,2.17,329.28,4316.667889,b,b,b,2053.247382


In [75]:
# Функция для добавления столбца со словами рост, снижение и без изменений
def alert2(row):
  if row['Разница в суммах между текущим и прошедшим периодами'] > 0:
    return 'Рост'
  elif row['Разница в суммах между текущим и прошедшим периодами'] < 0:
    return 'Снижение'
  else:
    return 'Без изменений'

In [76]:
df['Значение'] = df.apply(alert2, axis=1)

In [77]:
# Округляю значения в столбцах 
df['Сумма, руб. за текущий период'] = df['Сумма, руб. за текущий период'].round(0)
df['Сумма, руб. за предыдущий период'] = df['Сумма, руб. за предыдущий период'].round(0)
df['Разница в суммах между текущим и прошедшим периодами'] = df['Разница в суммах между текущим и прошедшим периодами'].round(0)
df['Количество за текущий период'] = df['Количество за текущий период'].round(2)

In [78]:
# Выгружаю данные для отчёта
df.to_excel('Витрина для отчёта abc-анализа покупателей.xlsx')